In [13]:
######################## 机器学习 CNN 类 ##############################
# Author: 杨玉奇
# email: yangyuqi@sina.com
# url: https://github.com/jerryyyq/tf_algorithm_example
# copyright yangyuqi
# 著作权归作者 杨玉奇 所有。商业转载请联系作者获得授权，非商业转载请注明出处。
# date: 2018-03-28
###################################################################

import tensorflow as tf

from common import *
from ML_Model import ML_Model
from img_to_tf_record import Img2TFRecord

class ML_CNN( ML_Model ):
    def __init__(self, feature_shape, label_number):
        # feature_number: 一个图像的 长 × 宽 × channels
        ML_Model.__init__(self)

        self.__label_number = label_number
        self.__feature_shape = feature_shape
        feature_shape.append(label_number)
        self._W = tf.Variable(tf.zeros(feature_shape), name = 'weights')
        self._b = tf.Variable(tf.zeros([label_number]), name = 'bias')
        self.__img_set = Img2TFRecord('/home/yangyuqi/Downloads/Images', '/tmp/tf_out/tmp1')


    def combine_inputs(self, features):
        return tf.matmul(features, self._W) + self._b

    #def inference(self, features):
        #return tf.nn.softmax( self.combine_inputs(features) )
    def inference(x, num_class):
        with tf.variable_scope('softmax'):
            dtype = x.dtype.base_dtype
            # Set up the requested initialization.
            init_mean = 0.0
            init_stddev = 0.0
            weights = tf.get_variable('weights',
                                [x.get_shape()[1], num_class], initializer=init_ops.random_normal_initializer(init_mean, init_stddev, dtype=dtype), dtype=dtype)
            biases = tf.get_variable('bias', [num_class], initializer=init_ops.random_normal_initializer(init_mean, init_stddev, dtype=dtype), dtype=dtype)

            logits = tf.nn.xw_plus_b(x, weights, biases)
            return logits

    
    def loss(self, features, label):
        label_predicted = self.combine_inputs(features)
        return tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(logits = label_predicted, labels = label) )
    
    
    def inputs(self, file_name = [], batch_size = 10):
        if file_name[0] == 'train':
            image_batch, label_batch = self.__img_set.read_train_images_from_tf_records([250, 151, 1], batch_size)
        else:
            image_batch, label_batch = self.__img_set.read_test_images_from_tf_records([250, 151, 1], batch)

        # image_batch2 = tf.reshape( image_batch, (batch_size, 250*151*1) )
        label_batch2 = tf.one_hot( label_batch, self.__label_number, 1, 0 )
        return image_batch, label_batch2
        
        
    def train(self, feature_batch, label_batch):

        conv2d_layer_one = tf.contrib.layers.convolution2d(
            feature_batch, 
            num_outputs = 32, 
            kernel_size = (5, 5), 
            activation_fn = tf.nn.relu,
            weights_initializer = tf.random_normal_initializer(),
            stride = (2, 2),
            trainable = True
        )

        pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
                                       ksize = [1, 2, 2, 1],
                                       strides = [1, 2, 2, 1],
                                       padding = 'SAME')

        print( conv2d_layer_one.get_shape() )
        print( pool_layer_one.get_shape() )


        conv2d_layer_two = tf.contrib.layers.convolution2d(
            pool_layer_one, 
            num_outputs = 64, 
            kernel_size = (5, 5), 
            activation_fn = tf.nn.relu,
            weights_initializer = tf.random_normal_initializer(),
            stride = (1, 1),
            trainable = True
        )

        pool_layer_two = tf.nn.max_pool(conv2d_layer_two,
                                       ksize = [1, 2, 2, 1],
                                       strides = [1, 2, 2, 1],
                                       padding = 'SAME')

        print( conv2d_layer_two.get_shape() )
        print( pool_layer_two.get_shape() )


        flattened_layer_two = tf.reshape( pool_layer_two, [feature_batch.get_shape().as_list()[0], -1] )
        print( flattened_layer_two.get_shape() )

        hidden_layer_three = tf.contrib.layers.fully_connected(
            flattened_layer_two, 
            512,
            weights_initializer = lambda i, dtype: tf.truncated_normal( [38912, 512], stddev = 0.1 ),
            activation_fn = tf.nn.relu
        )

        hidden_layer_three = tf.nn.dropout( hidden_layer_three, 0.1 )
        final_fully_connected = tf.contrib.layers.fully_connected(
            hidden_layer_three,
            120,
            weights_initializer = lambda i, dtype: tf.truncated_normal( [512, 120], stddev = 0.1 )
        )
        
        self._total_loss = self.loss( feature_batch, label_batch )

        learning_rate = 0.01
        return tf.train.GradientDescentOptimizer( learning_rate ).minimize( self._total_loss )

    
    def evaluate(self, test_features, test_label):
        echo_tensor(self._sess, self._W, 'At evaluate, the _W')
        echo_tensor(self._sess, self._b, 'At evaluate, the _b')
        echo_tensor(self._sess, test_features, 'At evaluate, test_features')
        echo_tensor(self._sess, test_label, 'At evaluate, test_label')
        
        label_predicted = tf.cast( tf.arg_max(self.inference(test_features), 1), tf.int32 )
        
        evaluate_result = tf.reduce_mean(tf.cast(tf.equal(label_predicted, test_label), tf.float32))
        evaluate_result = tf.Print( evaluate_result, [evaluate_result], '*********** evaluate_result: ' )
        return evaluate_result

    
if __name__ == '__main__':
    one_ml = ML_CNN([250, 151, 1], 65)
    # data from: https://archive.ics.uci.edu/ml/datasets/Iris
    one_ml.do_train( 10, ['train'] )  # 10000 次可以得到 80% 的准确率

    one_ml.do_evaluate( ['test'] )







-------------- do_train: start -----------------
(10, 125, 76, 32)
(10, 63, 38, 32)
(10, 63, 38, 64)
(10, 32, 19, 64)
(10, 38912)


ValueError: Shape (10, 250, 151, 1) must have rank 2

In [5]:
help(tf.contrib.layers.convolution2d)

Help on function convolution2d in module tensorflow.contrib.layers.python.layers.layers:

convolution2d(inputs, num_outputs, kernel_size, stride=1, padding='SAME', activation_fn=<function relu at 0x7f7bf146c840>, normalizer_fn=None, normalizer_params=None, weights_initializer=<function variance_scaling_initializer.<locals>._initializer at 0x7f7bf07ee620>, weights_regularizer=None, biases_initializer=<function zeros_initializer at 0x7f7bf15a4400>, biases_regularizer=None, reuse=None, variables_collections=None, outputs_collections=None, trainable=True, scope=None)
    Adds a 2D convolution followed by an optional batch_norm layer.
    
    `convolution2d` creates a variable called `weights`, representing the
    convolutional kernel, that is convolved with the `inputs` to produce a
    `Tensor` of activations. If a `normalizer_fn` is provided (such as
    `batch_norm`), it is then applied. Otherwise, if `normalizer_fn` is
    None and a `biases_initializer` is provided then a `biases` v

In [ ]:
import tensorflow as tf
from img_to_tf_record import Img2TFRecord

one_Set = Img2TFRecord('/home/yangyuqi/Downloads/Images', '/tmp/tf_out/tmp1')

image_batch, label_batch = one_Set.read_train_images_from_tf_records([250, 151, 1], 5)
init = tf.initialize_all_variables()   # tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init)

    coord = tf.train.Coordinator() 
    threads = tf.train.start_queue_runners( sess = sess, coord = coord )

    
         for i in range(3):
            img, lab = sess.run([image_batch, label_batch])
            print(img.shape, lab)

    
    conv2d_layer_one = tf.contrib.layers.convolution2d(
        image_batch, 
        num_outputs = 32, 
        kernel_size = (5, 5), 
        activation_fn = tf.nn.relu,
        weights_initializer = tf.random_normal_initializer,
        stride = (2, 2),
        data_format = 'NHWC',
        trainable = True
    )

    pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
                                   ksize = [1, 2, 2, 1],
                                   strides = [1, 2, 2, 1],
                                   padding = 'SAME')

    print( conv2d_layer_one.get_shape() )
    print( pool_layer_one.get_shape() )
    
    
    conv2d_layer_two = tf.contrib.layers.convolution2d(
        pool_layer_one, 
        num_outputs = 64, 
        kernel_size = (5, 5), 
        activation_fn = tf.nn.relu,
        weights_initializer = tf.random_normal_initializer,
        stride = (1, 1),
        trainable = True
    )

    pool_layer_two = tf.nn.max_pool(conv2d_layer_two,
                                   ksize = [1, 2, 2, 1],
                                   strides = [1, 2, 2, 1],
                                   padding = 'SAME')

    print( conv2d_layer_two.get_shape() )
    print( pool_layer_two.get_shape() )
    

    flattened_layer_two = tf.reshape( pool_layer_tow, [batch_size, -1] )
    print( flattened_layer_two.get_shape() )

    hidden_layer_three = tf.contrib.layers.fully_connected(
        flattened_layer_two, 
        512,
        weight_init = lambda i, dtype: tf.truncated_normal( [38912, 512], stddev = 0.1 ),
        activation_fn = tf.nn.relu
    )

    hidden_layer_three = tf.nn.dropout( hidden_layer_three, 0.1 )
    final_fully_connected = tf.contrib.layers.fully_connected(
        hidden_layer_three,
        120,
        weight_init = lambda i, dtype: tf.truncated_normal( [512, 120], stddev = 0.1 )
    )
    
    
    
    for step in range(1000):    # 实际训练闭环 
        if coord.should_stop():
            print('---- coord should stop ----')
            break

        sess.run([final_fully_connected])

        # 查看训练过程损失递减
        if step % 10 == 0:
            echo_tensor( self._sess, features, 'features_' + str(step) )
            echo_tensor( self._sess, label, 'label_' + str(step) )
            echo_tensor( self._sess, total_loss, 'step_' + str(step) + ' loss: ' )

            #print( str(training_steps) + " final loss: ", sess.run([total_loss]) )
            echo_tensor( self._sess, total_loss, 'training end. step_' + str(step) + ' final loss: ' )

            saver = tf.train.Saver()
            save_path = saver.save( self._sess, self.__save_path )
            print('save_path is: ', save_path)

            # 模型评估
            evaluate_result = self.evaluate( features, label ) 
            echo_tensor( self._sess, evaluate_result, 'evaluate_result' )   
    
    
    
    
    
    
    
            
            
            
            
    #关闭线程  
    coord.request_stop()  
    coord.join(threads)


